Authors:
Zach Baker - 23155062
Seb Matthews - 22482441

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import seaborn as sns

import os

# World Network Setup

Follows heavily from:
https://github.com/Tina333333/WorldCityNetwork/blob/main/CityLinesMinus.xlsx
Airports PNG from MyFlights.py

## Graphing
As based on MyFlights.py

more info at https://gallery.pyecharts.org/#/Map/china_gdp_from_1993_to_2018
https://pyecharts.org/#/en-us/series_options

In [128]:
# pip install pyecharts
from pyecharts.charts import *
from pyecharts import options as opts
from pyecharts.commons.utils import JsCode


world_map = Map(init_opts=opts.InitOpts(theme='white', bg_color='white', width='990px', height='540px'))

# itemstyle_opts=opts.ItemStyleOpts(color="#D3D3D3", border_color="#1E90FF"),
# emphasis_label_opts=opts.LabelOpts(is_show=False),
# emphasis_itemstyle_opts=opts.ItemStyleOpts(color="#323c48")


def add_population(world_map, pop_name, pop_data):
    # Pop Data should be like this:
    # pop_name: the birth place of population
    # pop_data:
    #   [[Current_Location_Name1, Count1], [Current_Location_Name2, Count2], ...]
    #   e.g. [["Australia", 1000], ["China", 2000]]

    # Test adding a simple series
    world_map.add(
        pop_name,
        pop_data,
        "world",
        is_roam=False,
        # tooltip_opts=opts.TooltipOpts(formatter=pop_name + ":{a} - {c}")
        # tooltip_opts=opts.TooltipOpts(formatter=f"{pop_name} in {{b}} - {{c}}\n", trigger='item')
        # tooltip_opts=opts.TooltipOpts(trigger='item', formatter = "Australians: {@[1]}")
    ).set_series_opts(label_opts=opts.LabelOpts(is_show=False)) # Remove labels
# world_m


add_population(world_map, 
    "Australians", 
    [["Australia", 1000], ["China", 200], ["India", 5]]
    )

add_population(world_map, 
    "Chinese", 
    [["Australia", 100], ["China", 30000], ["India", 500]]
    )

# add_population(world_map, 
#     "Population", 
#     [["Australia", {"Australians": 100, "Chinese": 2}],
#         ["China", {"Australians": 1, "Chinese": 2}], 
#         ["India", {"Australians": 1, "Chinese": 2}]]
#     )

# tooltip_formatter = JsCode("""
# function (params) {
#     let result = params[0].name + ':<br/>';
#     for (let i = 0; i < params.length; i++) {
#         result += '• ' + params[i].seriesName + ': ' + params[i].value + '<br/>';
#     }
#     return result;
# }
# """)

world_map.set_global_opts(
        # title_opts=opts.TitleOpts(title="Population"),
        # visualmap_opts=opts.VisualMapOpts(max_=2000, is_piecewise=True, series_index=1),
        visualmap_opts=opts.VisualMapOpts(max_=2000), # https://pyecharts.org/#/en-us/global_options?id=visualmapopts-visual-mapping-configuration-items
        # visualmap_opts=opts.VisualMapOpts(max_=2000, type_="size"),
        # tooltip_opts=opts.TooltipOpts(formatter="{}: {c}")
        # tooltip_opts=opts.TooltipOpts(trigger='series')
        tooltip_opts=opts.TooltipOpts(trigger='item')
        # legend_opts=opts.LegendOpts(
        #     selected_mode="multiple"  # optional: "single" to show one at a time
        #     ),
        # Split into comma separated list
        # tooltip_opts=opts.TooltipOpts(trigger='item', formatter = "Australians: {@Australians}")
    )


world_map.render_notebook()

# Agent Setup
Structure from Wk 7 lab

In [ ]:
# PLACEHOLDER - UPDATE LATER
class Agent:

    def __init__(self, loc, params):
        """Creates a new agent at the given location.

        loc: tuple coordinates
        params: dictionary of parameters
        """
        self.loc = tuple(loc)
        self.age = 0

        # extract the parameters
        max_vision = params.get('max_vision', 6) # PLACEHOLDER EXAMPLE

        # choose attributes: randomly choose
        self.vision = np.random.randint(1, max_vision+1) # PLACEHOLDER EXAMPLE

    def step(self, env):
        """Look around, move, and harvest.

        env: Sugarscape object
        """
        # returns the agent’s new location, which is the visible cell with the most sugar.
        self.loc = env.look_and_move(self.loc, self.vision)
        # takes the (new) location of the agent, and removes and returns the sugar at that location.
        # update the sugar of this agent after adding up the harvested sugar and minus the metabolism
        self.sugar += env.harvest(self.loc) - self.metabolism
        self.age += 1

    def is_starving(self):
        """Checks if sugar has gone negative."""
        return self.sugar < 0

    def is_old(self):
        """Checks if lifespan is exceeded."""
        return self.age > self.lifespan

# Simulation Class
Runs the step-by-step

In [ ]:
#PLACEHOLDER
class Sugarscape(Cell2D):
    """Represents an Epstein-Axtell Sugarscape."""

    def __init__(self, n, **params):
        """Initializes the attributes.

        n: number of rows and columns
        params: dictionary of parameters
        """
        self.n = n
        self.params = params # a dictionary where parameters are saved.

        # track variables
        self.agent_count_seq = []

        # make the capacity array
        self.capacity = self.make_capacity()

        # initially all cells are at capacity
        self.array = self.capacity.copy()

        # make the agents
        self.make_agents()

    def make_capacity(self):
        """Makes the capacity array."""

        # compute the distance of each cell from the peaks.
        dist1 = distances_from(self.n, 15, 15)
        dist2 = distances_from(self.n, 35, 35)
        dist = np.minimum(dist1, dist2)

        # cells in the capacity array are set according to dist from peak
        bins = [21, 16, 11, 6]
        a = np.digitize(dist, bins)
        return a

    def make_agents(self):
        """Makes the agents."""

        # determine where the agents start and generate locations
        n, m = self.params.get('starting_box', self.array.shape)
        locs = make_locs(n, m) # return the locations of each cell in a grid by indices of row and column
        np.random.shuffle(locs)

        # make the agents
        num_agents = self.params.get('num_agents', 400)
        assert(num_agents <= len(locs))
        # make a list of agents, with each has its own associated attributes: sugar, vision, metabolism etc.
        self.agents = [Agent(locs[i], self.params)
                       for i in range(num_agents)]

        # keep track of which cells are occupied
        self.occupied = set(agent.loc for agent in self.agents)

    def grow(self):
        """Adds sugar to all cells and caps them by capacity."""
        grow_rate = self.params.get('grow_rate', 1)
        self.array = np.minimum(self.array + grow_rate, self.capacity) # the total sugar in each cell is bounded by its capacity

    def look_and_move(self, center, vision):
        """Finds the visible cell with the most sugar.

        center: tuple, coordinates of the center cell
        vision: int, maximum visible distance

        returns: tuple, coordinates of best cell
        """
        # find all visible cells
        locs = make_visible_locs(vision)
        locs = (locs + center) % self.n

        # convert rows of the array to tuples
        locs = [tuple(loc) for loc in locs]

        # select unoccupied cells
        empty_locs = [loc for loc in locs if loc not in self.occupied]

        # if all visible cells are occupied, stay put
        if len(empty_locs) == 0:
            return center

        # look up the sugar level in each cell
        t = [self.array[loc] for loc in empty_locs]

        #return the location with the highest sugar level
        i = np.argmax(t)
        return empty_locs[i]

    def harvest(self, loc):
        """Removes and returns the sugar from `loc`.

        loc: tuple coordinates
        """
        sugar = self.array[loc]
        self.array[loc] = 0
        return sugar

    def step(self):
        """Executes one time step."""
        replace = self.params.get('replace', False)

        # loop through the agents in random order
        random_order = np.random.permutation(self.agents)
        for agent in random_order:

            # mark the current cell unoccupied
            self.occupied.remove(agent.loc)

            # execute one step that updates the agent's new location and sugar level
            agent.step(self)

            # if the agent is dead, remove from the list
            if agent.is_starving() or agent.is_old():
                self.agents.remove(agent)
                if replace:
                    self.add_agent()
            else:
                # otherwise mark its cell occupied
                self.occupied.add(agent.loc)

        # update the time series
        self.agent_count_seq.append(len(self.agents))

        # grow back some sugar
        self.grow()
        return len(self.agents)

    def add_agent(self):
        """Generates a new random agent.

        returns: new Agent
        """
        new_agent = Agent(self.random_loc(), self.params)
        self.agents.append(new_agent)
        self.occupied.add(new_agent.loc)
        return new_agent

    def random_loc(self):
        """Choose a random unoccupied cell.

        returns: tuple coordinates
        """
        while True:
            loc = tuple(np.random.randint(self.n, size=2))
            if loc not in self.occupied:
                return loc

    def draw(self):
        """Draws the cells."""
        draw_array(self.array, cmap='YlOrRd', vmax=9, origin='lower')

        # draw the agents
        xs, ys = self.get_coords()
        self.points = plt.plot(xs, ys, '.', color='red')[0]

    def get_coords(self):
        """Gets the coordinates of the agents.

        Transforms from (row, col) to (x, y).

        returns: tuple of sequences, (xs, ys)
        """
        agents = self.agents
        rows, cols = np.transpose([agent.loc for agent in agents])
        xs = cols + 0.5
        ys = rows + 0.5
        return xs, ys

# Comparison to dataset
Placeholder for later analysis

In [ ]:
os.path.join("datasets", "bilat_mig.xlsx")

# STATS
Placeholder for later analysis